In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math
import datetime
import warnings

from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics.scorer import make_scorer

from bayes_opt import BayesianOptimization

warnings.filterwarnings(action='ignore')


plt.rcParams['font.family'] = 'NanumBarunGothic'
%config inlinebackend.figure_format = 'retina'
plt.rcParams['figure.dpi'] = 100 

## 1. 데이터 전처리

In [ ]:
train = pd.read_csv('train.csv', encoding='cp949')
test = pd.read_csv('test.csv', encoding='cp949')
test = test.rename(columns={'강수량(mm, 6시간)':'강수량(mm)',
                    '일조(hr, 3시간)':'일조(hr)'})
train['date_time'] = pd.to_datetime(train['date_time'])
test['date_time'] = pd.to_datetime(test['date_time'])


sample_submission = pd.read_csv('sample_submission.csv', encoding='cp949')

print(f'Train: {train.shape}')
print(f'Test: {test.shape}')
print(f'Submission: {sample_submission.shape}')

### 1-1. 결측치 처리

- train 데이터의 기간동안 비전기냉방설운영 형태나 태양광보유 상황이 달라지지 않은 것으로 보여, 현재의 값들로 test 데이터의 Null값을 채웁니다.
- 기온/풍속/습도/강수령/일조데이터는 선형보간으로 값을 채웁니다.

In [ ]:
sun = train.groupby('num')['태양광보유'].unique()
non_ele = train.groupby('num')['비전기냉방설비운영'].unique()

In [ ]:
test['태양광보유'] = test['num'].map(sun).astype(int)
test['비전기냉방설비운영'] = test['num'].map(non_ele).astype(int)

In [ ]:
test = test.interpolate()
test.head()

### 1-2. 날짜 변수 전처리 및 공휴일 데이터는 노이즈라고 판단하여 제거

In [ ]:
# hour, month 등 변수 추가
train['hour'] = train['date_time'].dt.hour
train['month'] = train['date_time'].dt.month
train['day'] = train['date_time'].dt.day
train['weekday'] = train['date_time'].dt.weekday
train['date'] = train['date_time'].dt.date

test['hour'] = test['date_time'].dt.hour
test['month'] = test['date_time'].dt.month
test['day'] = test['date_time'].dt.day
test['weekday'] = test['date_time'].dt.weekday
test['date'] = test['date_time'].dt.date

In [ ]:
# 주 및 다음날 변수 추가
train['week'] = train['date_time'].dt.isocalendar().week
train['next_week'] = train['week']+1
train['tomorrow_day'] = pd.to_datetime(train['date'] + datetime.timedelta(days=1)).dt.day
train['next_hour'] = (train['hour'] + 1).replace({24:0})

test['week'] = test['date_time'].dt.isocalendar().week
test['next_week'] = test['week']+1
test['tomorrow_day'] = pd.to_datetime(test['date'] + datetime.timedelta(days=1)).dt.day
test['next_hour'] = (test['hour'] + 1).replace({24:0})

In [ ]:
# 1번: 6/1, 6/2, 6/3, 6/4
train = train[~((train['num'] == 1) & (train['month'] == 6) & train['day'].isin([1,2,3,4]))]
    
# 2번: 8/17
train = train[~((train['num'] == 2) & (train['month'] == 8) & train['day'].isin([17]))]

# 3번: 7/15~8/4이 아닌 광복절만 제거
train = train[~((train['num'] == 3) & (train['month'] == 8) & train['day'].isin([17]))]

# 6번: 8/17
train = train[~((train['num'] == 6) & (train['month'] == 8) & train['day'].isin([17]))]
    
# 7번: 8/17
train = train[~((train['num'] == 7) & (train['month'] == 8) & train['day'].isin([17]))]

# 8번: 8/17
train = train[~((train['num'] == 8) & (train['month'] == 8) & train['day'].isin([17]))]

# 9번: 6/3, 7/16
train = train[~((train['num'] == 9) & (train['month'] == 6) & train['day'].isin([3]))]
train = train[~((train['num'] == 9) & (train['month'] == 7) & train['day'].isin([16]))]

# 10번: 7/27, 8/10
train = train[~((train['num'] == 10) & (train['month'] == 7) & train['day'].isin([27]))]
train = train[~((train['num'] == 10) & (train['month'] == 8) & train['day'].isin([10]))]

# 13번: 8/17
train = train[~((train['num'] == 13) & (train['month'] == 8) & train['day'].isin([17]))]

# 16번: 8/17
train = train[~((train['num'] == 16) & (train['month'] == 8) & train['day'].isin([17]))]

# 17번: 8/17
train = train[~((train['num'] == 17) & (train['month'] == 8) & train['day'].isin([17]))]

# 18번: 8/17
train = train[~((train['num'] == 18) & (train['month'] == 8) & train['day'].isin([17]))]

# 22번: 8/17
train = train[~((train['num'] == 22) & (train['month'] == 8) & train['day'].isin([17]))]

# 23번: 8/17
train = train[~((train['num'] == 23) & (train['month'] == 8) & train['day'].isin([17]))]

# 24번: 6/1
train = train[~((train['num'] == 24) & (train['month'] == 6) & train['day'].isin([1]))]
    
# 25번: 7/27~7/31, 8/17
train = train[~((train['num'] == 25) & (train['month'] == 7) & train['day'].isin([27, 28, 29, 30, 31]))]
train = train[~((train['num'] == 25) & (train['month'] == 8) & train['day'].isin([17]))]

# 26번: 8/17
train = train[~((train['num'] == 26) & (train['month'] == 8) & train['day'].isin([17]))]

# 27번: 8/17
train = train[~((train['num'] == 27) & (train['month'] == 8) & train['day'].isin([17]))]

# 33번: 8/17
train = train[~((train['num'] == 33) & (train['month'] == 8) & train['day'].isin([17]))]

# 35번: 8/17
train = train[~((train['num'] == 35) & (train['month'] == 8) & train['day'].isin([17]))]

# 37번: 8/17
train = train[~((train['num'] == 37) & (train['month'] == 8) & train['day'].isin([17]))]

# 38번: 8/17
train = train[~((train['num'] == 38) & (train['month'] == 8) & train['day'].isin([17]))]

# 40번: 8/3
train = train[~((train['num'] == 40) & (train['month'] == 8) & train['day'].isin([3]))]

# 42번: 7/13, 8/10
train = train[~((train['num'] == 42) & (train['month'] == 7) & train['day'].isin([13]))]
train = train[~((train['num'] == 42) & (train['month'] == 8) & train['day'].isin([10]))]

# 43번: 8/17
train = train[~((train['num'] == 43) & (train['month'] == 8) & train['day'].isin([17]))]

# 44번: 8/17
train = train[~((train['num'] == 44) & (train['month'] == 8) & train['day'].isin([17]))]

# 45번: 6/6, 8/15
train = train[~((train['num'] == 45) & (train['month'] == 6) & train['day'].isin([6]))]
train = train[~((train['num'] == 45) & (train['month'] == 8) & train['day'].isin([15]))]
    
# 46번: 6/6, 8/17
train = train[~((train['num'] == 46) & (train['month'] == 6) & train['day'].isin([6]))]
train = train[~((train['num'] == 46) & (train['month'] == 8) & train['day'].isin([17]))]    

# 47번: 8/17
train = train[~((train['num'] == 47) & (train['month'] == 8) & train['day'].isin([17]))]

# 48번: 8/17
train = train[~((train['num'] == 48) & (train['month'] == 8) & train['day'].isin([17]))]

# 52번: 8/17
train = train[~((train['num'] == 52) & (train['month'] == 8) & train['day'].isin([17]))]

# 53번: 8/17
train = train[~((train['num'] == 53) & (train['month'] == 8) & train['day'].isin([17]))]

# 54번: 6/5, 8/17
train = train[~((train['num'] == 54) & (train['month'] == 6) & train['day'].isin([5]))]
train = train[~((train['num'] == 54) & (train['month'] == 8) & train['day'].isin([17]))]

# 55번: 8/17
train = train[~((train['num'] == 55) & (train['month'] == 8) & train['day'].isin([17]))]

# 56번: 8/17
train = train[~((train['num'] == 56) & (train['month'] == 8) & train['day'].isin([17]))]

# 57번: 6/6, 8/15, 8/17
train = train[~((train['num'] == 57) & (train['month'] == 6) & train['day'].isin([6]))]
train = train[~((train['num'] == 57) & (train['month'] == 8) & train['day'].isin([15, 17]))]

# 58번: 8/17
train = train[~((train['num'] == 58) & (train['month'] == 8) & train['day'].isin([17]))]

### 1-3. 군집화
 - 건물에 따른 일별 그리고 요일별 전력사용량 산출
 - 각 전력사용량을 표준화 실시 (일별, 요일별을 따로 하기 위해 sklearn의 StandardScaling 사용하지 않음)
 - elbow method를 활용한 최적 군집수 파악 및 군집화

In [ ]:
# 건물을 기준으로 하는 data frame 생성
by_weekday = train.groupby(['num','weekday'])['전력사용량(kWh)'].median().reset_index().pivot('num','weekday','전력사용량(kWh)').reset_index()
by_hour = train.groupby(['num','hour'])['전력사용량(kWh)'].median().reset_index().pivot('num','hour','전력사용량(kWh)').reset_index().drop('num', axis = 1)
df_cluster = pd.concat([by_weekday, by_hour], axis= 1)
columns = ['num'] + ['day'+str(i) for i in range(7)] + ['hour'+str(i) for i in range(24)]
df_cluster.columns = columns

# standard scaling
for i in range(len(df_cluster)):
    df_cluster.iloc[i,1:8] = (df_cluster.iloc[i,1:8] - df_cluster.iloc[i,1:8].mean())/df_cluster.iloc[i,1:8].std() # 요일 별 전력 중앙값에 대해 scaling
    df_cluster.iloc[i,8:] = (df_cluster.iloc[i,8:] - df_cluster.iloc[i,8:].mean())/df_cluster.iloc[i,8:].std() # 시간대별 전력 중앙값에 대해 scaling
    
df_cluster

In [ ]:
# elbow method를 통해 군집의 개수 결정
def change_n_clusters(n_clusters, data):
    sum_of_squared_distance = []
    for n_cluster in n_clusters:
        kmeans = KMeans(n_clusters=n_cluster)
        kmeans.fit(data)
        sum_of_squared_distance.append(kmeans.inertia_)
        
    plt.figure(1 , figsize = (12, 6))
    plt.plot(n_clusters , sum_of_squared_distance , 'o')
    plt.plot(n_clusters , sum_of_squared_distance , '-' , alpha = 0.5)
    plt.xlabel('Number of Clusters')
    plt.ylabel('Inertia')

change_n_clusters([2,3,4,5,6,7,8,9,10,11], df_cluster.iloc[:,1:]) # 최적군집수: 4

In [ ]:
# 건물별 군집화 결과
OPTIMAL_CLUSTER = 4

kmeans = KMeans(n_clusters=OPTIMAL_CLUSTER, random_state = 2)
df_cluster['cluster'] = kmeans.fit_predict(df_cluster.iloc[:,1:])
df_cluster_result = df_cluster[['num', 'cluster']].set_index('num')
df_cluster_result['cluster'] = df_cluster_result['cluster'].replace({0:9999}).replace({1:8888}).replace({2:7777}).replace({3:6666})
df_cluster_result['cluster'] = df_cluster_result['cluster'].replace({9999:1}).replace({8888:3}).replace({7777:2}).replace({6666:0})

In [ ]:
# 적용
train['cluster'] = train['num'].map(df_cluster_result['cluster']) # map을 쓰려면 index를 merge key로 지정 후 Series형태로 바꿔줄 것 
test['cluster'] = test['num'].map(df_cluster_result['cluster']) # map을 쓰려면 index를 merge key로 지정 후 Series형태로 바꿔줄 것 

In [ ]:
# 군집화 결과 체크
fig = plt.figure(figsize = (20, 4))
for c in range(4):
    temp = train[train.cluster == c]
    temp = temp.groupby(['weekday', 'hour'])['전력사용량(kWh)'].median().reset_index().pivot('weekday', 'hour', '전력사용량(kWh)')
    plt.subplot(1, 5, c+1)
    sns.heatmap(temp)
    plt.title(f'cluster {c}')
    plt.xlabel('')
    plt.ylabel('')
    plt.yticks([])

### 1-4. 불쾌지수

In [ ]:
def DI(temp, humid):

    return 0.81 * temp +0.01 * humid * (0.99*temp - 14.3)+ 46.3


# DI 80 이상이면 50% 정도 불쾌감을 느낌
train['DI'] = DI(train['기온(°C)'], train['습도(%)'])
train['feel_DI'] = train['DI'].map(lambda x : 1 if x > 80 else 0)

test['DI'] = DI(test['기온(°C)'], train['습도(%)'])
test['feel_DI'] = test['DI'].map(lambda x : 1 if x > 80 else 0)

### 1-5. 태양광

In [ ]:
train['태양광'] =  train['일조(hr)'] * train['태양광보유'] * train['강수량(mm)'].map(lambda x : np.exp(-x)) * train['풍속(m/s)'] / train['습도(%)'] * 100
test['태양광'] =  test['일조(hr)'] * test['태양광보유'] * test['강수량(mm)'].map(lambda x : np.exp(-x)) * test['풍속(m/s)'] / test['습도(%)'] * 100

### 1-6. 체감온도

In [ ]:
def get_pow(series):
    return math.pow(series, 0.15)

train['perceived_temperature'] = 13.12 + 0.6215*train['기온(°C)'] - 11.37*train['풍속(m/s)'].apply(get_pow) + 0.3965*train['풍속(m/s)'].apply(get_pow)*train['기온(°C)']
test['perceived_temperature'] = 13.12 + 0.6215*test['기온(°C)'] - 11.37*test['풍속(m/s)'].apply(get_pow) + 0.3965*test['풍속(m/s)'].apply(get_pow)*test['기온(°C)']

### 1-7. 최저/최고온도

In [ ]:
# 그날의 최저기온 & 최고기온
train = train.merge(train.groupby(['num','date'])['기온(°C)'].min().reset_index().rename(columns = {'기온(°C)':'min_temperature'}), on = ['num','date'], how = 'left')
test = test.merge(test.groupby(['num','date'])['기온(°C)'].min().reset_index().rename(columns = {'기온(°C)':'min_temperature'}), on = ['num','date'], how = 'left')

train = train.merge(train.groupby(['num','date'])['기온(°C)'].max().reset_index().rename(columns = {'기온(°C)':'max_temperature'}), on = ['num','date'], how = 'left')
test = test.merge(test.groupby(['num','date'])['기온(°C)'].max().reset_index().rename(columns = {'기온(°C)':'max_temperature'}), on = ['num','date'], how = 'left')

train.head()

### 1-8. 근무시간 및 점심시간 변수 추가

In [ ]:
train['working_time'] = 0
test['working_time'] = 0
train['lunch_time'] = 0
test['lunch_time'] = 0

train.loc[(train['cluster'] == 3) & (train['hour'].isin([8,9,10,11,12,13,14,15,16,17,18,19])), 'working_time'] = 1
train.loc[(train['cluster'] == 2) & (train['hour'].isin([18,19,20,21,22])), 'working_time'] = 1
train.loc[(train['cluster'] == 1) & (train['hour'].isin([8,9,10,11,12,13,14,15,16,17,18,19])) & (train['weekday'].isin([0,1,2,3,4])), 'working_time'] = 1
train.loc[((train['cluster'] == 1) & (train['hour'].isin([11,12,13])) & (train['weekday'].isin([0,1,2,3,4]))), 'lunch_time'] = 1

test.loc[(test['cluster'] == 3) & (test['hour'].isin([8,9,10,11,12,13,14,15,16,17,18,19])), 'working_time'] = 1
test.loc[(test['cluster'] == 2) & (test['hour'].isin([18,19,20,21,22])), 'working_time'] = 1
test.loc[(test['cluster'] == 1) & (test['hour'].isin([8,9,10,11,12,13,14,15,16,17,18,19])) & (test['weekday'].isin([0,1,2,3,4])), 'working_time'] = 1
test.loc[(test['cluster'] == 1) & (test['hour'].isin([11,12,13])) & (test['weekday'].isin([0,1,2,3,4])), 'lunch_time'] = 1

### 1-9. 평일/주말 구분 변수 생성

In [ ]:
def weekend(day):
    if day >= 5:
        return 1
    else:
        return 0

train['weekend_bool'] = train['weekday'].apply(lambda x: weekend(x))
test['weekend_bool'] = test['weekday'].apply(lambda x: weekend(x))

# 추가적으로 공휴일을 고려해준다. 8/15의 대체공휴일인 8/17 --> 공휴일로 취급
train.loc[(train['month'] == 8) & (train['day'] == 17), 'weekend_bool'] = 1

### 1-10. 월과 일을 결합한 파생변수

In [ ]:
# 월과 일수를 합쳐 파생변수 생성
train['day_count'] = 99999
test['day_count'] = 99999

train.loc[train['month'] == 6, 'day_count'] = train['month'] * 30 + train['day']
train.loc[~(train['month'] == 6), 'day_count'] = train['month'] * 31 + train['day']

test.loc[~(test['month'] == 6), 'day_count'] = test['month'] * 31 + test['day']

train['day_count_yesterday'] = train['day_count'].shift(1).fillna(method = 'backfill')
train['day_count_tomorrow'] = train['day_count'].shift(-1).fillna(method = 'ffill')
test['day_count_yesterday'] = test['day_count'].shift(1).fillna(method = 'backfill')
test['day_count_tomorrow'] = test['day_count'].shift(-1).fillna(method = 'ffill')

In [ ]:
# 월과 일수를 합쳐 파생변수 생성
train['hour_count'] = 99999
test['hour_count'] = 99999

train.loc[train['month'] == 6, 'hour_count'] = train['month'] * 30 + train['day'] + train['hour'] / 24
train.loc[~(train['month'] == 6), 'hour_count'] = train['month'] * 31 + train['day'] + train['hour'] / 24

test.loc[~(test['month'] == 6), 'hour_count'] = test['month'] * 31 + test['day'] + train['hour'] / 24

train['hour_count_yesterday'] = train['hour_count'].shift(1).fillna(method = 'backfill')
train['hour_count_tomorrow'] = train['hour_count'].shift(-1).fillna(method = 'ffill')
test['hour_count_yesterday'] = test['hour_count'].shift(1).fillna(method = 'backfill')
test['hour_count_tomorrow'] = test['hour_count'].shift(-1).fillna(method = 'ffill')

### 1-11. 냉방도일

In [ ]:
# CDH feature추가
def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

In [ ]:
cdhs = np.array([])
for num in range(1,61,1):
    temp = train[train['num'] == num]
    cdh = CDH(temp['기온(°C)'].values)
    cdhs = np.concatenate([cdhs, cdh])
train['CDH'] = cdhs


cdhs = np.array([])
for num in range(1,61,1):
    temp = test[test['num'] == num]
    cdh = CDH(temp['기온(°C)'].values)
    cdhs = np.concatenate([cdhs, cdh])
test['CDH'] = cdhs

### 1-12. 태양광

In [ ]:
train['태양광'] =  train['일조(hr)'] * train['태양광보유'] * train['강수량(mm)'].map(lambda x : np.exp(-x)) * train['풍속(m/s)'] / train['습도(%)'] * 100
test['태양광'] =  test['일조(hr)'] * test['태양광보유'] * test['강수량(mm)'].map(lambda x : np.exp(-x)) * test['풍속(m/s)'] / test['습도(%)'] * 100

### 1-13 1시간, 2시간 전 상황 고려

In [ ]:
train['기온_1시간전'] = train['기온(°C)'].shift(1)
train['기온_2시간전'] = train['기온(°C)'].shift(2)
train['습도_1시간전'] = train['습도(%)'].shift(1)
train['습도_2시간전'] = train['습도(%)'].shift(2)
train['강수량_1시간전'] = train['강수량(mm)'].shift(1)
train['강수량_2시간전'] = train['강수량(mm)'].shift(2)
train['불쾌지수_1시간전'] = train['DI'].shift(1)
train['불쾌지수_2시간전'] = train['DI'].shift(2)
train['체감불쾌지수_1시간전'] = train['feel_DI'].shift(1)
train['체감불쾌지수_2시간전'] = train['feel_DI'].shift(2)
train['체감온도_1시간전'] = train['perceived_temperature'].shift(1)
train['체감온도_2시간전'] = train['perceived_temperature'].shift(2)


test['기온_1시간전'] = test['기온(°C)'].shift(1)
test['기온_2시간전'] = test['기온(°C)'].shift(2)
test['습도_1시간전'] = test['습도(%)'].shift(1)
test['습도_2시간전'] = test['습도(%)'].shift(2)
test['강수량_1시간전'] = test['강수량(mm)'].shift(1)
test['강수량_2시간전'] = test['강수량(mm)'].shift(2)
test['불쾌지수_1시간전'] = test['DI'].shift(1)
test['불쾌지수_2시간전'] = test['DI'].shift(2)
test['체감불쾌지수_1시간전'] = test['feel_DI'].shift(1)
test['체감불쾌지수_2시간전'] = test['feel_DI'].shift(2)
test['체감온도_1시간전'] = test['perceived_temperature'].shift(1)
test['체감온도_2시간전'] = test['perceived_temperature'].shift(2)

train = train.fillna(method = 'backfill')
test = test.fillna(method = 'backfill')

### 1-14. 코로나 컬럼 추가

In [ ]:
month=np.array(train.month)
day=np.array(train.day)

corona=[]
for m,d in zip(month,day):
    if m==8 and d>=16:
        corona.append(2)
    else:
        corona.append(1)
corona2=[]
for i in range(len(test)):
    corona2.append(2)
train['corona']=corona
test['corona']=corona2

### 1-15 누적강수시간

In [ ]:
def cumulative_rain_hour(idx, data, data_type):
    count = 0

    while True:
        
        # 0일때만 예외처리
        if (data_type == 'train') & (idx == 0):
            count = 1
            break
            
        elif (data_type == 'test') & (idx == 0):
            count = 0
            break
            
        
        if data.loc[idx-count, '강수량(mm)'] == 0:
            break
            
        else:
            count += 1
            
        if idx-count <= 0:
            count += 1
            break
            
    return count

train['누적강수시간'] = [cumulative_rain_hour(idx, train, 'train') for idx in range(train.shape[0])]
test['누적강수시간'] = [cumulative_rain_hour(idx, test, 'test') for idx in range(test.shape[0])]

### 1-16. 일평균일조량

In [ ]:
train = pd.merge(train, train.groupby('date')['일조(hr)'].agg({'mean'}), how='left', on='date').rename(columns={'mean':'일평균일조량'})
test = pd.merge(test, test.groupby('date')['일조(hr)'].agg({'mean'}), how='left', on='date').rename(columns={'mean':'일평균일조량'})

### 1-17. 폭염지수

In [ ]:
final_pokyeom = []
for j in range(1,61):
    a = list(train[train.num == j].groupby('date')['기온(°C)'].max().values)
    b = []
    for i in range(len(a)-1):
        if a[i] >= 35:
            if a[i+1] >= 35:
                b.append(2)
            elif a[i+1] >= 33:
                b.append(1)
            else:
                b.append(0)
        elif 33 <= a[i] <= 35:
            if a[i+1] >= 33:
                b.append(1)
            else:
                b.append(0)
        else:
            b.append(0)


    b.append(b[-1])
    for k in b:
        for o in range(24):
            final_pokyeom.append(k)
            
            
train['폭염주의_폭염경보'] = final_pokyeom

In [ ]:
final_pokyeom = []
for j in range(1,61):
    a = list(test[test.num == j].groupby('date')['기온(°C)'].max().values)
    b = []
    for i in range(len(a)-1):
        if a[i] >= 35:
            if a[i+1] >= 35:
                b.append(2)
            elif a[i+1] >= 33:
                b.append(1)
            else:
                b.append(0)
        elif 33 <= a[i] <= 35:
            if a[i+1] >= 33:
                b.append(1)
            else:
                b.append(0)
        else:
            b.append(0)


    b.append(b[-1])
    for k in b:
        for o in range(24):
            final_pokyeom.append(k)
            
test['폭염주의_폭염경보'] = final_pokyeom

### 1-18. 표준편차 변수 추가

In [ ]:
# 60개의 건물 그룹화
num_groups = train.groupby("num")

# 건물 번호 매칭을 위해 빈 리스트 하나 추가 후 진행
group_list = [[]]
for i in range(1,61):
    group_list.append(num_groups.get_group(i))

# 건물별 표준편차(168)개 담을 DataFrame 생성
std_df=pd.DataFrame()

for num in range(1,60+1):
    std_df = std_df.append((group_list[num][:2040-24].pivot_table(index='weekday',columns='hour', values='전력사용량(kWh)', aggfunc='std')/group_list[num][:2040-24]['전력사용량(kWh)'].median()).mul(100).unstack().to_frame().T)

std_df['num'] = range(1,60+1)
std_df.set_index('num',inplace=True)

In [ ]:
std_df2 = std_df.stack(0).stack(0).reset_index().rename(columns={0:'std'})
train = train.merge(std_df2, on=['num','hour','weekday'], how='inner')
test = test.merge(std_df2, on=['num','hour','weekday'], how='inner')

### 1-19. 중앙값 변수 추가

In [ ]:
# 건물별 중앙값(168)개 담을 DataFrame 생성
median_df=pd.DataFrame()

for num in range(1,60+1):

  # 건물별 
  median_df = median_df.append(group_list[num][:2040-24].pivot_table(index='weekday',columns='hour', values='전력사용량(kWh)', aggfunc='median').unstack().to_frame().T)

median_df['num'] = range(1,60+1)
median_df.set_index('num',inplace=True)

In [ ]:
# 피처 생성
median_df = median_df.unstack().rename("median_effect")
train = train.merge(median_df, how='left', left_on=['hour','weekday', 'num'], right_index=True)
test = test.merge(median_df, how='left', left_on=['hour','weekday', 'num'], right_index=True)

### 1-20. OnehotEncoding(weekday, hour)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# 정의
enc = OneHotEncoder()

# train
object_col = ['hour','weekday','day', 'month']

enc.fit(train.loc[:,object_col])

train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
#train.drop(object_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)

# test
test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test.reset_index(drop=True, inplace=True)
#test.drop(object_col, axis=1, inplace=True)
test = pd.concat([test, test_onehot_df], axis=1)

### 1-21. 주말평일+시간별 전력사용량 변수 추가

In [ ]:
df_tmp = pd.DataFrame()

for num in range(1,61):
    tmp = train[train['num'] == num]
    tmp = (tmp.groupby(['weekend_bool','hour'])['전력사용량(kWh)'].mean() / tmp['전력사용량(kWh)'].median()).reset_index().rename(columns={'전력사용량(kWh)':'주말평일_시간별전력량'})
    tmp['num'] = num
    
    df_tmp = df_tmp.append(tmp, ignore_index=True)
    
train = train.merge(df_tmp, on=['num','weekend_bool','hour'], how='left')
test = test.merge(df_tmp, on=['num','weekend_bool','hour'], how='left')

### 1-22 .주별 전력사용량 표준편차 및 평균

In [ ]:
# 주별 전력사용량 표준편차 및 평균 --> 건물별 적합 실시 (마지막단계에 해보자)

for num in range(1,61):
    
    week_mean_dict = train[train['num']==num].groupby('week')['전력사용량(kWh)'].mean()
    week_std_dict = train[train['num']==num].groupby('week')['전력사용량(kWh)'].std()
    
    train.loc[train['num'] == num, 'week_mean'] = train['week'].map(week_mean_dict)
    test.loc[test['num'] == num, 'week_mean'] = test['week'].map(week_mean_dict)
    train.loc[train['num'] == num, 'week_std'] = train['week'].map(week_std_dict)
    test.loc[test['num'] == num, 'week_std'] = test['week'].map(week_std_dict)

### 1-23. 시간 --> cosine encoding

In [ ]:
train['hour'] = train['hour'].astype('int')
test['hour'] = test['hour'].astype('int')

train['cos_hour'] = np.cos(2*np.pi*train['hour']/24)
test['cos_hour'] = np.cos(2*np.pi*test['hour']/24)

### 1-9999. 빼야할 Column은 배제하기

In [ ]:
drop_cols = ['date_time', 'date', 'week_mean', 'week_std'] # week_mean과 week_std는 test에서 null값이 존재하기에 배제
train.drop(drop_cols, axis=1, inplace=True)
test.drop(drop_cols, axis=1, inplace=True)

## 2. Modeling
- K-fold로 모델을 구성 및 Stacking 앙상블 실시합니다

In [ ]:
X = train.drop(['전력사용량(kWh)'], axis = 1) # 건물별 적합하기에 모두 같은값들은 제거하자.
y = train['전력사용량(kWh)']

X_test = test.copy()

### 1. 파라미터 찾기

In [ ]:
def SMAPE_RF(y_pred, y_true):
 
    v = 2 * abs(y_pred - y_true) / (abs(y_pred) + abs(y_true))
    SMAPE = - (np.mean(v) * 100)
    
    return SMAPE

In [ ]:
def rf_evaluate(max_depth, min_samples_split, min_samples_leaf):
    params = {
        'max_depth': int(max_depth),
        'min_samples_split' : int(round(min_samples_split)),
        'min_samples_leaf' : int(round(min_samples_leaf))}
    
    RF = RandomForestRegressor(**params,random_state = 42, n_estimators = 500)
    kf = KFold(n_splits=5, random_state=42, shuffle=True)
    scores = cross_val_score(RF, X, y, cv=kf, scoring=make_scorer(SMAPE_RF, greater_is_better=True))
    
    return np.mean(scores)

In [ ]:
rf_params = {}

for BUILDING_NUM in range(1,61):
    
    print(f'\n\n================== 건물명: {BUILDING_NUM} 시작!! ==================')

    tmp = train[train['num'] == BUILDING_NUM].drop('num', axis=1).reset_index(drop=True)
    X = tmp.drop(['전력사용량(kWh)'], axis = 1)
    X = X.drop(['비전기냉방설비운영', '태양광보유', 'cluster'], axis = 1) # 건물별로 동일한 값은 제거
    X[['weekday','week','next_week']] = X[['weekday','week','next_week']].astype('int')
    y = tmp['전력사용량(kWh)']

    rfBO = BayesianOptimization(rf_evaluate, 
                             {'max_depth': (5,100),
                             'min_samples_split':(5,100),
                             'min_samples_leaf': (1,50),
                              },
                              random_state=42)

    rfBO.maximize(init_points=5, n_iter=30) # 30

    rf_param = rfBO.max['params']
    rf_param['learning_rate'] = 0.01
    rf_params[BUILDING_NUM] = rf_param

### 2. 모델에 대입

In [ ]:
def smape(A, F):
    
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [ ]:
n_splits = 5
rf_models={}
model_name = []
valid_prediction = []
valid_label = []
valid_SMAPE = []

for BUILDING_NUM in range(1,61):
    
    print(f'\n\n================== 건물명: {BUILDING_NUM} 시작!! ==================')

    step = 1
    cv = KFold(n_splits = n_splits, shuffle = True, random_state=42)
    
    tmp = train[train['num'] == BUILDING_NUM].drop('num', axis=1).reset_index(drop=True)
    tmp = tmp.drop(['비전기냉방설비운영', '태양광보유', 'cluster'], axis = 1) # 건물별로 동일한 값은 제거
    X = tmp.drop(['전력사용량(kWh)'], axis = 1)
    X[['weekday','week','next_week']] = X[['weekday','week','next_week']].astype('int')
    y = tmp['전력사용량(kWh)']
    
    rf_final_param = {
                      "min_samples_leaf" : 10,
                      "min_samples_split" : 10,
                      "max_depth": -1
                      }
    
    rf_final_param['min_samples_leaf'] = int(rf_params[BUILDING_NUM]['min_samples_leaf'])
    rf_final_param['min_samples_split'] = int(rf_params[BUILDING_NUM]['min_samples_split'])
    rf_final_param['max_depth'] = rf_params[BUILDING_NUM]['max_depth']
    
    for tr_idx, val_idx in cv.split(X):
        print(f'\n\n ============================ {step} ============================')
        
        X_train = X.iloc[tr_idx, :].values
        y_train = y[tr_idx].values

        X_valid = X.iloc[val_idx, :].values
        y_valid = y[val_idx].values

        rf_model = RandomForestRegressor(**rf_final_param, random_state = 42, n_estimators = 200)
        rf_model.fit(X_train,y_train)
        
        # 모델저장
        dict_key = f'{BUILDING_NUM}_{step}'
        rf_models[dict_key] = rf_model
        
        # 데이터프레임 적재
        model_name.append(dict_key)
        valid_prediction.append(rf_model.predict(X_valid))
        valid_label.append(y_valid.tolist())
        valid_SMAPE.append(smape(y_valid, rf_model.predict(X_valid)))
        
        
        # log & step += 1
        print(f'SMAPE: {round(smape(y_valid, rf_model.predict(X_valid)), 5)}')
        step += 1
    
df_rf_predict = pd.DataFrame({'모델명':model_name, 'y_pred':valid_prediction, 'y_true':valid_label, 'SMAPE':valid_SMAPE})

In [ ]:
# 4, 15, 18, 34, 42, 59 --> 훈련이상해,,

# Submission

In [ ]:
X_test[['hour','weekday','week','next_week']] = X_test[['hour','weekday','week','next_week']].astype('int')
X_test.dtypes

In [ ]:
answer_list = []
model_name = []
test_prediction = []

for BUILDING_NUM in range(1,61):
    
    if BUILDING_NUM % 10 == 0:
        print(f'== 건물명: {BUILDING_NUM} 시작!! ==')
    
    values = []
    tmp = X_test[X_test['num'] == BUILDING_NUM].reset_index(drop=True).drop('num', axis=1)
    tmp = tmp.drop(['비전기냉방설비운영', '태양광보유', 'cluster'], axis = 1) # 건물별로 동일한 값은 제거

    for step in range(1,1+n_splits):

        dict_key = f'{BUILDING_NUM}_{step}'
        value = pd.Series(rf_models[dict_key].predict(tmp) / n_splits)
        
        if step == 1:
            values = value.copy()
        else:
            values += value
        
        # 데이터프레임에 적재
        model_name.append(dict_key)
        test_prediction.append(rf_models[dict_key].predict(tmp).tolist())
    
    answer_list.extend(values.tolist())
df_rf_predict2 = pd.DataFrame({'모델명':model_name, 'y_pred':test_prediction})

In [ ]:
df_rf_predict = df_rf_predict.rename(columns={'y_pred':'valid_pred','y_true':'valid_true','SMAPE':'vaild_SMAPE'})
df_rf_predict2 = df_rf_predict2.rename(columns={'y_pred':'test_pred'})
df_rf_predict_concat = pd.merge(df_rf_predict, df_rf_predict2, on='모델명')
df_rf_predict_concat.to_csv('df_rf_prediction.csv', index=False, encoding='cp949')

In [ ]:
submission = sample_submission.copy()
submission['answer'] = answer_list
submission.to_csv('submission_0622(XGB_스태킹_앙상블).csv', index= False) 